In [1]:
#This code has been inspired from blog https://towardsdatascience.com/q-learning-54b841f3f9e4
import pandas as pd
import random
import numpy as np
import time

In [2]:
class Game:
    rewards=None
    positionCol=None
    positionRow=None
    
    def __init__(self,startCol=1,startRow=1):
        self.rewards=pd.DataFrame({1:[0,1,2,3,4],2:[1,2,3,4,5],3:[2,3,4,5,6],4:[3,4,5,6,7],5:[4,5,6,7,8]},index={1,2,3,4,5})
        self.positionCol=startCol
        self.positionRow=startRow
        
    def move(self,direction):
        reward=0
        end=False
        if direction=='Up':
            self.positionRow-=1
        elif direction=='Down':
            self.positionRow+=1
        elif direction=='Left':
            self.positionCol-=1
        else:
            self.positionCol+=1
        
        #check if we lost.
        if self.positionRow<1 or self.positionRow>5 or self.positionCol<1 or self.positionCol>5:
            end=True
            reward=-1000
        #Check if we reached the end.
        elif self.positionCol==5 and self.positionRow==5:
            end=True
            reward=self.rewards[self.positionCol][self.positionRow]
        else:
            end=False
            reward=self.rewards[self.positionCol][self.positionRow]
            
        return (reward,end)
    
    

#Learning part.
learning_rate=1
discount=0
random_explore=0.1
qtable=pd.DataFrame(100,index=['Up','Down','Left','Right'],columns=[11,12,13,14,15,21,22,23,24,25,31,32,33,34,35,41,42,43,44,45,51,52,53,54,55])

for i in range(1000):
    game=Game()
    end_of_game=False
    while not end_of_game:
        current_state=(game.positionCol*10)+game.positionRow
        max_reward_action=qtable[current_state].idxmax()
        if random.random()<random_explore:
            max_reward_action=qtable.index[random.randint(0,3)]
        reward,end_of_game=game.move(max_reward_action)
        if end_of_game:
            qtable.loc[max_reward_action,current_state]=reward
        else:
            optimal_future_value=qtable[(game.positionCol*10)+game.positionRow].max()
            discounted_optimal_future_value=discount*optimal_future_value
            learning_value=reward+discounted_optimal_future_value
            qtable.loc[max_reward_action,current_state]=(1-learning_rate)*qtable[current_state][max_reward_action]+learning_rate*learning_value
            
        

In [3]:
def traverse_path_gui(Col,Row):
    
    end_game=False
    first_move=1
    while end_game is False:
        
        move=qtable[(Col*10)+Row].idxmax()
        prevCol=Col
        prevRow=Row
        
        if move=='Up':
            Row-=1
        elif move=='Down':
            Row+=1
        elif move=='Left':
            Col-=1
        else:
            Col+=1
        if Row<1 or Row>5 or Col<1 or Col>5:
            end_game=True
        #Check if we reached the end.
        elif Col==5 and Row==5:
            end_game=True
        if first_move:
            #Do not remove earlier ticks.
            print("Row=%d Col=%f"%(Row,Col))
            button_array[Row-1][Col-1]["text"]="X"
            time.sleep(1)
            
            
        else:
            #Remove earlier marks and place new one.
            print("Row=%d Col=%f"%(prevRow,prevCol))
            button_array[prevRow-1][prevCol-1]["text"]=""
            print("Row=%d Col=%f"%(Row,Col))
            button_array[Row-1][Col-1]["text"]="X"
            time.sleep(1)



            
def get_next_move(Col,Row):
    end_game=False
    move=qtable[(Col*10)+Row].idxmax()
    if move=='Up':
        Row-=1
    elif move=='Down':
        Row+=1
    elif move=='Left':
        Col-=1
    else:
        Col+=1
    if Row<1 or Row>5 or Col<1 or Col>5:
        end_game=True
    #Check if we reached the end.
    elif Col==5 and Row==5:
        end_game=True
        
    return Col,Row,end_game

def reach_the_destination():
    #Lets initially obtain the move for first move.
    Col,Row,game_ended=get_next_move(1,1)
    while game_ended is not True:
        #Lets display the tick.
        button_array[Col-1][Row-1]["text"]="X"
        ui.mainloop()
        Col,Row,game_ended=get_next_move(Col,Row)
        
        
        

        
    
    
    
        
        
        

In [4]:
from tkinter import *
ui=Tk()
ui.title("RL Agent Navigating the board")

button_array=[[0 for i in range(5)] for i in range(5)]
for i in range(5):
    for j in range(5):
        button_array[i][j]=Button(ui,text="",bg='gray',fg='white',width=10,height=5)
        button_array[i][j].grid(row=i,column=j)
        
    

#Start=Button(ui,bg='gray',fg='white',text="start",command=lambda:reach_the_destination())
Start=Button(ui,bg='gray',fg='white',text="start",command=lambda:traverse_path_gui(1,1))
Start.grid(row=6,column=2)
ui.mainloop()



Row=2 Col=1.000000
Row=3 Col=1.000000
Row=4 Col=1.000000
Row=5 Col=1.000000
Row=5 Col=2.000000
Row=5 Col=3.000000
Row=5 Col=4.000000
Row=5 Col=5.000000
